# Introduction
This notebook is use for made for doing a capstone project for coursera course.

In [0]:
import pandas as pd
import numpy as np

# What we want to achieve in this section
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [2]:
import urllib3
from bs4 import BeautifulSoup
http = urllib3.PoolManager()
r = http.request('GET', 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.data, 'lxml')

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


soup object contains all the source code for wiki page

In [0]:
# table class is "wikitable sortable jquery-tablesorter"
table = soup.find('table',{'class':'wikitable sortable'})

Extracting table into a 2d list which will be transformed in Dataframe

In [0]:
table_list = []
for tr in table.find_all('tr')[1:]:
    row_list = []
    for tds in tr.find_all('td'):
        row_list.append(tds.text)
    table_list.append(row_list)

In [0]:
Dataframe = pd.DataFrame(table_list, columns=["Postalcode","Borough","Neighborhood"])

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [0]:
Dataframe = Dataframe[Dataframe["Borough"] != 'Not assigned\n']

In [7]:
Dataframe.head()

,Postalcode,Borough,Neighborhood
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n
5,M6A\n,North York\n,Lawrence Manor / Lawrence Heights\n
6,M7A\n,Downtown Toronto\n,Queen's Park / Ontario Provincial Government\n


In [0]:
# replace all \n with ''
 Dataframe['Borough'] = Dataframe['Borough'].str.replace('\n','')
 Dataframe['Postalcode'] = Dataframe['Postalcode'].str.replace('\n','')
 Dataframe['Neighborhood'] = Dataframe['Neighborhood'].str.replace('\n','')
 Dataframe['Neighborhood'] = Dataframe['Neighborhood'].str.replace('/',',')


In [9]:
Dataframe.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In my case there was no Neighborhood column with value "Not Assigned"
but i am writing the code for it.


In [0]:
Dataframe.loc[Dataframe['Neighborhood'] == 'Not assigned', 'Neighborhood'] =Dataframe['Borough']

In [11]:
Dataframe.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


* checking is there any postal code which come twice
* if so next cell we merge them into one.

In [12]:
a = Dataframe.groupby('Postalcode').count()
a['Neighborhood'].sort_values(ascending=False)

Postalcode
M9W    1
M4E    1
M3C    1
M3H    1
M3J    1
      ..
M6A    1
M6B    1
M6C    1
M6E    1
M1B    1
Name: Neighborhood, Length: 103, dtype: int64

In [0]:

Dataframe = Dataframe.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)


## Final Data frame

In [14]:
Dataframe=Dataframe.reset_index()
Dataframe.head(15)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District , Ryerson"
